# Download Meteor M2 satellite info and calculate times for recording

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(httr)

In [3]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [4]:
library(jsonlite)


Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    flatten



In [5]:
library(assertthat)


Attaching package: ‘assertthat’

The following object is masked from ‘package:tibble’:

    has_name



# Helper functions

In [6]:
possibly_get_data <- function(url, n_tries, ...){

    rate <- rate_backoff(pause_base = 1, max_times = n_tries)
    possibly_insistent_get <- insistently(GET, rate, quiet = FALSE) %>% possibly(otherwise = NULL)

    possibly_insistent_get(url, ...)
}

# Config

What satellite are we trying to capture?

In [7]:
satellite_id <- "METEOR-M2"

In [8]:
satellite_norad_id <- 40069

In [9]:
n2yo_api_key <- "&apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ"

In [10]:
geonames_username <- "pgcudahy"

Minimal elevation of pass to capture satellite (in degrees)

In [11]:
minimum_observable_elevation = 20

Station longitude, latitude. Positive values for E, negative for W

In [12]:
station_latitude <- c(-29.53)
station_longitude <- c(30.25)

Sanity check that your coordinates are close to a nearby city

In [13]:
nearby_cities <- possibly_get_data(paste0("http://api.geonames.org/findNearbyPlaceNameJSON?lat=", station_latitude, 
                                   "&lng=", station_longitude, "&username=", geonames_username), 8)

In [14]:
content(nearby_cities)$geonames[[1]]$toponymName

[1] "Merrivale"

In [15]:
content(nearby_cities)$geonames[[1]]$countryName

[1] "South Africa"

And get the elevation (in meters above sea level)

In [16]:
station_elevation <- possibly_get_data(paste0("http://api.geonames.org/gtopo30JSON?lat=", station_latitude, 
                                   "&lng=", station_longitude, "&username=", geonames_username), 8)

In [17]:
(station_elevation <- content(station_elevation)$gtopo30)

[1] 1167

# Get satellite orbital data

In [18]:
#satellite_data <- possibly_get_data("http://www.celestrak.com/NORAD/elements/weather.txt", 5)

In [19]:
#satellite_data

In [20]:
#satellite_df <- content(satellite_data) %>% strsplit("\r\n", fixed=TRUE) %>% unlist %>% 
#    matrix(ncol = 3, byrow = TRUE, dimnames=list(NULL, c("satellite", "a", "b"))) %>% as_tibble

In [21]:
#satellite_df %>% filter(grepl(satellite_id, satellite))

## Let's try this again using the https://www.n2yo.com/api/ webservice since I can't find any R packages to predict satellite passes

Request: /radiopasses/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{days}/{min_elevation}
```
Parameter       Type	Required	Comments
id              integer Yes	        NORAD id
observer_lat	float   Yes	        Observer's latitide (decimal degrees format)
observer_lng	float   Yes	        Observer's longitude (decimal degrees format)
observer_alt	float   Yes	        Observer's altitude above sea level in meters
days            integer Yes	        Number of days of prediction (max 10)
min_elevation   integer Yes	        The minimum elevation acceptable for the highest altitude point of the pass (degrees)
```

In [22]:
paste("https://www.n2yo.com/rest/v1/satellite/radiopasses",
                                          satellite_norad_id, station_latitude, station_longitude,
                                          station_elevation, 10, minimum_observable_elevation, n2yo_api_key,
                                          sep="/")

[1] "https://www.n2yo.com/rest/v1/satellite/radiopasses/40069/-29.53/30.25/1167/10/20/&apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ"

In [23]:
satellite_data <- possibly_get_data(paste("https://www.n2yo.com/rest/v1/satellite/radiopasses",
                                          satellite_norad_id, station_latitude, station_longitude,
                                          station_elevation, 10, minimum_observable_elevation, n2yo_api_key,
                                          sep="/"), 8)

In [24]:
satellite_content <- fromJSON(rawToChar(satellite_data$content))

In [25]:
assert_that(!is.null(satellite_content$passes),
        msg = paste("\nUnable to updated orbital data from n2yo.com",
                   "\nstatus code:", satellite_data$status_code, 
                   "\nerror messge:", satellite_content$error))

[1] TRUE

In [26]:
satellite_data_df <- satellite_content %>% pluck("passes") %>% as_tibble

In [27]:
assert_that(nrow(satellite_data_df) > 0,
           msg = "Updated orbital data from n2yo.com is empty")

[1] TRUE

In [28]:
satellite_data_df %<>%
    mutate(startUTC = as_datetime(startUTC, tz = "UTC")) %>%
    mutate(endUTC = as_datetime(endUTC, tz = "UTC")) %>%
    mutate(startDate = format(startUTC, "%Y-%m-%d")) %>%
    mutate(startTime = format(startUTC, "%H:%M")) %>%
    mutate(endDate = format(endUTC, "%Y-%m-%d")) %>%
    mutate(endTime = format(endUTC, "%H:%M")) %>%
    mutate(localStartTime = format(as_datetime(startUTC, tz = "Africa/Johannesburg") - 60, "%H:%M")) %>% 
    mutate(duration = as.numeric(hm(format(format="%H:%M", endTime)) -
                                hm(format(format="%H:%M",startTime))) / 60)

Note: method with signature ‘Period#ANY’ chosen for function ‘-’,
 target signature ‘Period#Period’.
 "ANY#Period" would also be valid


In [29]:
head(satellite_data_df)

startAz,startAzCompass,startUTC,maxAz,maxAzCompass,maxEl,maxUTC,endAz,endAzCompass,endUTC,startDate,startTime,endDate,endTime,localStartTime,duration
<dbl>,<chr>,<dttm>,<dbl>,<chr>,<dbl>,<int>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
184.19,S,2019-09-14 19:34:05,248.18,WSW,24.77,1568490060,314.95,NW,2019-09-14 19:47:55,2019-09-14,19:34,2019-09-14,19:47,21:33,13
40.75,NE,2019-09-15 05:10:15,109.46,ESE,28.83,1568524645,177.55,S,2019-09-15 05:24:30,2019-09-15,05:10,2019-09-15,05:24,07:09,14
345.85,N,2019-09-15 06:50:20,277.36,W,27.08,1568530645,208.04,SSW,2019-09-15 07:04:35,2019-09-15,06:50,2019-09-15,07:04,08:49,14
178.29,S,2019-09-15 19:13:55,252.09,WSW,38.58,1568575280,326.32,NW,2019-09-15 19:28:35,2019-09-15,19:13,2019-09-15,19:28,21:12,15
357.16,N,2019-09-16 06:30:00,278.49,W,43.17,1568615850,201.37,SSW,2019-09-16 06:45:00,2019-09-16,06:30,2019-09-16,06:45,08:29,15
172.58,S,2019-09-16 18:53:50,252.28,WSW,61.83,1568660490,337.23,NNW,2019-09-16 19:09:00,2019-09-16,18:53,2019-09-16,19:09,20:52,16


In [30]:
dates <- summarise(satellite_data_df, min(startDate, na.rm=TRUE),
          max(startDate, na.rm=TRUE)) %>% paste(collapse="_")

In [31]:
saveRDS(satellite_data_df, paste0("satellite_data_", dates, ".rds"))